In [1]:
%load_ext autoreload
%autoreload 2
import yaml
import pandas as pd
# from pandas.core.common import SettingWithCopyWarning
# pd.options.mode.chained_assignment = None
# import warnings

# warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
import numpy as np
import os
opj = os.path.join
opd = os.path.dirname
import glob
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm
from utils import *
from analyse import *
 
Z = norm.ppf

In [2]:
letters=np.load('/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-004/sub-004_ses-0_task-attnE_run-2_Logs/sub-004_ses-0_task-attnE_run-2_rsvp_letters20231122100652.npy')
# oris=np.load('/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/0/0_ses-0_task-attnE_run-0_Logs/0_ses-0_task-attnE_run-0_rsvp_ori20231120094047.npy')
# colors=np.load('/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/0/0_ses-0_task-attnH_run-0_Logs/0_ses-0_task-attnH_run-0_rsvp_color20231107143812.npy')

In [7]:
out='sub-006_ses-1_task-attnE_run-6'
sub=out.split('_')[0]
tsk=out.split('_')[2].split('-')[-1]
attn=out.split('_')[2][-1].lower()
name='train'

# substr =['ses-1','T_run']
# tr_f = [f.rsplit('_',1)[0] for f in os.listdir(opj(os.getcwd(),f'logs/{sub}')) if (substr[0] in f) & (substr[1] in f)]

beh = AnalyseRSVP(out, 'rsvp', attn,sub,name,verbose=False)
df=beh.loadRunLog()
beh.analyseYesNo()

Settings file:  /data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-006/sub-006_ses-1_task-attnE_run-6_Logs/sub-006_ses-1_task-attnE_run-6_expsettings.yml

Attention E\Target(s): ['x', 'X']
Response Keys: ['nan' 'b']

Off task results
h D': 0.463, C: 1.900

21 expected responses                        
17 actual subject responses                        
1 hits (within 1s)                        
20 misses                        
16 false alarms                        
951 correct rejections                        
Average RT: 1.439s

On task results
e D': 3.543, C: 1.097

20 expected responses                        
17 actual subject responses                        
15 hits (within 1s)                        
5 misses                        
2 false alarms                        
967 correct rejections                        
Average RT: 0.704s



In [16]:
exp_folder=opj(os.environ['PATH_HOME'],'pRF_difficulty/Experiment')

fnames=glob.glob(opj(exp_folder,'logs/sub-005/*'))
[fnames.append(glob.glob(opj(exp_folder,'logs/sub-006/*'))[i]) for i in range(len(glob.glob(opj(exp_folder,'logs/sub-006/*'))))]
fnames


['/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-005/sub-005_ses-1_task-attnE_run-2_Logs',
 '/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-005/sub-005_ses-1_task-attnH_run-4_Logs',
 '/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-005/sub-005_ses-1_task-attnE_run-3_Logs',
 '/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-005/sub-005_ses-1_task-attnH_run-1_Logs',
 '/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-005/sub-005_ses-1_task-attnH_run-3_Logs',
 '/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-005/sub-005_ses-1_task-attnE_run-1_Logs',
 '/data1/projects/dumoulinlab/Lab_members/Sumiya/programs/packages/pRF_difficulty/Experiment/logs/sub-005/sub-005_ses-1_task-attnH_run-2

In [ ]:
beh_df = pd.DataFrame()
exp_folder=opj(opd(os.environ['PATH_HOME']),'pRF_difficulty/Experiment')
fnames=glob.glob(opj(exp_folder,'logs/sub-005/*'))
[fnames.append(glob.glob(opj(exp_folder,'logs/sub-006/*'))[i]) for i in range(len(glob.glob(opj(exp_folder,'logs/sub-006/*'))))]

for folder in fnames:
    if 'attnT' in folder:
        continue
    # elif excl in folder:
    #     continue
    out = folder.rsplit('/',1)[1].rsplit('_',1)[0]
    sub=out.split('_')[0]
    tsk=out.split('_')[2].split('-')[-1]
    attn=out.split('_')[2][-1].lower()
    name='exp'
    f = f'{os.getcwd()}/logs/{sub}/{folder}/{folder.rsplit("_",1)[0]}_events.tsv'
    beh = AnalyseRSVP(out, 'rsvp', attn,sub,name,wd=exp_folder,verbose=False)
    df = pd.read_table(f,keep_default_na=True)
    df['subject'] = sub
    df['run'] = out.split('_')[3][-1]
    df['cond'] = out.split('_')[2][-1]
    beh_df = pd.concat((beh_df,df))

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(5,5),sharey=True,sharex=True)
jitter=0
for sub in beh_df.subject.unique():
    jitter+=0.05
    for cond in ['E','H']:
        x=0 if cond == 'E' else 1
        for run in beh_df.run.unique():
            df=beh_df.loc[(beh_df.subject==sub)&(beh_df.cond==cond) & (beh_df.run==run)& (beh_df.event_type == 'response')]
            # print(f'{cond}, Run {run}, Correct: {df.performance.sum()/225*100:.2f}%')
            ax.scatter(x+jitter,df.performance.sum()/225,marker=settings['markers'][sub],c=settings['cmap'][sub])

ax.set_xticks([0,1],['Easy','Hard'])
ax.set_ylabel('% Correct')
fig.tight_layout()